In [17]:
!pip3 install python-Levenshtein

In [3]:
import pymorphy2

In [44]:
import psycopg2
import Levenshtein
import pandas as pd
import random
from tqdm import tqdm
import json

In [39]:
import re

In [156]:
from Levenshtein import distance

In [ ]:
SELECT word_id, word_lemma, word_rating, jdesc->>'wd' AS word_value
FROM public.content_words
WHERE(word_hash != 0) AND (((array_length(regexp_split_to_array(content_words.jdesc->>'wd', '[ ''-]'), 1) = 1))) and word_rating > 100
ORDER BY word_rating DESC
LIMIT 100 OFFSET 0;

In [30]:
distance("go","goes")

2

In [13]:
conn = psycopg2.connect(dbname='pgstage', user='linguist', password='eDQGK0GCStlYlHNV', host='192.168.122.183')
cursor = conn.cursor()

interval = 10
offset = 0
request = """SELECT word_id, word_lemma, word_rating, jdesc->>'wd' AS word_value
    FROM public.content_words
WHERE(word_hash != 0) AND (((array_length(regexp_split_to_array(content_words.jdesc->>'wd', '[ ''-]'), 1) = 1))) and word_rating > 100
ORDER BY word_rating DESC LIMIT """ + str(interval) + " OFFSET " + str(offset)



In [14]:
print(request)
cursor.execute(request)

SELECT word_id, word_lemma, word_rating, jdesc->>'wd' AS word_value
    FROM public.content_words
WHERE(word_hash != 0) AND (((array_length(regexp_split_to_array(content_words.jdesc->>'wd', '[ ''-]'), 1) = 1))) and word_rating > 100
ORDER BY word_rating DESC LIMIT 10 OFFSET 0


In [15]:
for row in cursor:
    print(row)

(28174, 28174, 4492415, 'one')
(17769, 17769, 4472071, 'get')
(24966, 24966, 4283073, 'many')
(41009, 41009, 3388078, 'think')
(8864, 8864, 3223858, 'come')
(22725, 22725, 3186759, 'just')
(41282, 41282, 2986240, 'time')
(36143, 36143, 2929577, 'see')
(40307, 40307, 2827114, 'take')
(27615, 27615, 2761699, 'now')


In [19]:
unigraamm_db = pd.read_csv("top_10k_unigram.csv")
unigraamm_db.head()

,translate,trans_rating,translation_id,word_id,ref_id
0,мачеха,1617763,1,38937,1
1,супруг(а),1472268,1,38517,1
2,семья,1402033,1,15470,1
3,пасынок,1338321,1,38943,1
4,падчерица,1263808,1,38935,1


In [89]:
from tqdm import tqdm
import nltk

In [96]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /Users/nigula/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [99]:
tkn = nltk.word_tokenize("mother")
nltk.pos_tag(tkn)

[('mother', 'NN')]

In [4]:
morph = pymorphy2.MorphAnalyzer()

In [43]:
re.findall("[a-zA-Z0-9]","мааывsd33as")

['s', 'd', '3', '3', 'a', 's']

In [37]:
p = morph.parse('ахилл')[0]
print(p.normal_form, p.tag.POS)

ахилла NOUN


In [33]:
popular_unigramms = list(unigraamm_db['translate'])
letter_count_dict = {}
for word_ind in tqdm(range(len(popular_unigramms))):
    word = popular_unigramms[word_ind]
    tr_id = unigraamm_db.iloc[word_ind]['translation_id']
    word_id = unigraamm_db.iloc[word_ind]['word_id']
    p = morph.parse(word)[0]
    lemma = p.normal_form
    pos = p.tag.POS 
    """
    tkn = nltk.word_tokenize(word)
    word_pos = nltk.pos_tag(tkn)[0]
    wordnet_pos = get_wordnet_pos(word_pos[1]) 
    if (wordnet_pos):
        lemma = lemmatizer.lemmatize(word, pos = wordnet_pos)
    else:
        lemma = lemmatizer.lemmatize(word)
    """
    if len(word) in letter_count_dict:
        letter_count_dict[len(word)].append((word.lower(),pos,lemma,int(word_id),int(tr_id)))
    else:
        letter_count_dict[len(word)] = []
        letter_count_dict[len(word)].append((word.lower(),pos,lemma,int(word_id),int(tr_id)))


100%|██████████| 10000/10000 [00:07<00:00, 1318.55it/s]


def get_wordnet_pos(treebank_tag):

    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return ''

popular_unigramms = list(unigraamm_db['word_value'])
letter_count_dict = {}
for word_ind in tqdm(range(len(popular_unigramms))):
    word = popular_unigramms[word_ind]
    try:
        tkn = nltk.word_tokenize(word)
        word_pos = nltk.pos_tag(tkn)[0]
        wordnet_pos = get_wordnet_pos(word_pos[1]) 
        if (wordnet_pos):
            lemma = lemmatizer.lemmatize(word, pos = wordnet_pos)
        else:
            lemma = lemmatizer.lemmatize(word)
        if len(word) in letter_count_dict:
            letter_count_dict[len(word)].append((word_pos[0],word_pos[1],lemma))
        else:
            letter_count_dict[len(word)] = []
            letter_count_dict[len(word)].append((word_pos[0],word_pos[1],lemma))
        #print(word)
    except:
        pass

In [29]:
letter_count_dict[5]

[('семья', 'NOUN', 'семья', 15470, 1),
 ('отчим', 'NOUN', 'отчим', 38936, 1),
 ('стена', 'NOUN', 'стен', 44605, 1),
 ('мюсли', 'NOUN', 'мюсли', 7498, 1),
 ('жизнь', 'NOUN', 'жизнь', 23836, 1),
 ('сепия', 'NOUN', 'сепия', 36453, 1),
 ('белый', 'ADJF', 'белый', 45271, 1),
 ('ковер', 'NOUN', 'ковёр', 7108, 1),
 ('слово', 'NOUN', 'слово', 45681, 1),
 ('ванна', 'NOUN', 'ванна', 4568, 1),
 ('веник', 'NOUN', 'веник', 6259, 1),
 ('яркий', 'ADJF', 'яркий', 6176, 1),
 ('дверь', 'NOUN', 'дверь', 13065, 1),
 ('школа', 'NOUN', 'школа', 35741, 1),
 ('умный', 'ADJF', 'умный', 8312, 1),
 ('через', 'PREP', 'через', 41140, 1),
 ('полка', 'NOUN', 'полка', 36788, 1),
 ('ласка', 'NOUN', 'ласка', 7067, 1),
 ('свеча', 'NOUN', 'свеча', 6881, 1),
 ('ответ', 'NOUN', 'ответ', 3003, 1),
 ('белье', 'NOUN', 'бельё', 23978, 1),
 ('мысль', 'NOUN', 'мысль', 41056, 1),
 ('право', 'NOUN', 'право', 34664, 1),
 ('время', 'NOUN', 'время', 41282, 1),
 ('знать', 'INFN', 'знать', 23052, 1),
 ('радио', 'NOUN', 'радио', 33141, 

In [119]:
import json

In [ ]:
  print("\nNOW SAVING", file_name,'\n')
    with open(file_name, 'w', encoding = "utf-8") as outfile:
        json.dump(json_file, outfile, indent=1, separators=(',', ':'),ensure_ascii=False)

In [34]:
with open("unigram_pos_lemma_rus.json","w") as f:
    json.dump(letter_count_dict, f, indent = 1, ensure_ascii=False)

In [125]:
with open("unigram_pos.json","r") as f:
    data = json.load(f)
    data

In [144]:
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
lemmatizer = WordNetLemmatizer()

In [154]:
print("parental :", lemmatizer.lemmatize("parental", pos ="a")) 
print("gone :", lemmatizer.lemmatize("gone", pos ="v")) 


parental : parental
gone : go


In [107]:
distance("NN","NNS")

1

In [132]:
"parent" in "parents"

True

In [155]:
def get_wordnet_pos(treebank_tag):

    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return ''
    
def iterate_insde_dict(collected_words_list, word, examined_word_len, letter_count_dict,pos, search_range):
    collected_words = 0
    if examined_word_len in letter_count_dict and examined_word_len > 2:
        compare_words = letter_count_dict[examined_word_len]
        random.shuffle(compare_words)
        for word_compare_el in compare_words:
            word_compare = word_compare_el[0]
            pos_compare = word_compare_el[1]
            lemm_comapre = word_compare_el[2]
            if search_range > 3:
                pos_search_range = search_range
            else:
                pos_search_range = 1
            wordnet_pos = get_wordnet_pos(pos) 
            if (wordnet_pos):
                lemma = lemmatizer.lemmatize(word, pos = wordnet_pos)
            else:
                lemma = lemmatizer.lemmatize(word)
            if distance(lemm_comapre, lemma) > 0 and distance(lemm_comapre, lemma)  <= search_range and distance(pos_compare, pos) <= pos_search_range:
                collected_words_list.append(word_compare)
                collected_words += 1
                if (collected_words > 20):break
    return collected_words

def look_inside_dict(word, collected_words_list, word_len, pos, search_range):
    print (word, "word_len", word_len, "pos", pos, "search range", search_range)
    total_words = 0
    new_words_collected = iterate_insde_dict(collected_words_list, word, word_len-1, letter_count_dict, pos, search_range)
    total_words += new_words_collected
    new_words_collected = iterate_insde_dict(collected_words_list, word, word_len, letter_count_dict, pos, search_range)
    total_words += new_words_collected
    new_words_collected = iterate_insde_dict(collected_words_list, word, word_len+1, letter_count_dict, pos, search_range)
    total_words += new_words_collected
    if total_words < 25:
        look_inside_dict(word, collected_words_list, word_len, pos, search_range + 1)
    collected_words_set = set(collected_words_list)
    return collected_words_set
    
def look_for_similar ():
    output_unigramm_json = []
    conn = psycopg2.connect(dbname='pgstage', user='linguist', password='eDQGK0GCStlYlHNV', host='192.168.122.183')
    cursor_iterate = conn.cursor()
    interval = 20
    offset = 140
    request = """SELECT word_id, word_lemma, word_rating, jdesc->>'wd' AS word_value
    FROM public.content_words WHERE(word_hash != 0) AND (((array_length(regexp_split_to_array(content_words.jdesc->>'wd', '[ ''-]'), 1) = 1))) and word_rating > 100
    ORDER BY word_rating DESC LIMIT """ + str(interval) + " OFFSET " + str(offset)
    cursor = conn.cursor()
    cursor.execute(request)
    sim_words_dict = {}
    for row in cursor:
        curr_json = {}
        word_id = row[0]
        print(word_id)
        word = row[3]
        word_len = len(word)
        tkn = nltk.word_tokenize(word)
        pos = nltk.pos_tag(tkn)[0][1]
        word_container = []
        sim_words_dict[word] = look_inside_dict(word, word_container, word_len,pos,1)
        curr_json = {"word_id":word_id, "word":word, "simlar_words": sim_words_dict[word]}
        output_unigramm_json.append(curr_json)
        print(curr_json)
look_for_similar()
    

19038
happen word_len 6 pos VB search range 1
happen word_len 6 pos VB search range 2
happen word_len 6 pos VB search range 3
{'word_id': 19038, 'word': 'happen', 'simlar_words': {'hatred', 'playpen', 'appear', 'deepen', 'halved', 'aping', 'ripen', 'harping', 'hazing', 'aspen', 'applied', 'opened', 'shaped', 'happily', 'sharpen', 'appears', 'dappled', 'shaven', 'reopen', 'tapered', 'haired', 'dampen', 'haring', 'pigpen', 'lapsed', 'having', 'wakened', 'penned', 'shaping', 'ripened', 'apply'}}
44589
walk word_len 4 pos NN search range 1
walk word_len 4 pos NN search range 2
{'word_id': 44589, 'word': 'walk', 'simlar_words': {'sales', 'wards', 'bask', 'walls', 'milk', 'talk', 'elk', 'waif', 'hall', 'yanks', 'backs', 'ilk', 'falls', 'alt', 'gal', 'ask', 'talks', 'wax', 'lacks', 'als', 'parks', 'palo', 'pal', 'ali', 'tack', 'wave', 'dark', 'wad', 'balk', 'welts', 'tasks', 'wane', 'wales', 'salt', 'wild', 'wil', 'wacko', 'sacks', 'war', 'way', 'wale', 'hacks', 'masks', 'wick', 'wag', 'warm'

world word_len 5 pos NN search range 2
{'word_id': 45725, 'word': 'world', 'simlar_words': {'wards', 'worm', 'worn', 'lord', 'work', 'folds', 'ward', 'wooed', 'worry', 'colds', 'wordy', 'mould', 'wool', 'wolf', 'lords', 'wound', 'molds', 'whorl', 'ford', 'gold', 'cord', 'wolves', 'cold', 'wood', 'fold', 'told', 'wild', 'whorls', 'swords', 'weld', 'mold', 'woods', 'woulda', 'worth', 'sword', 'wield', 'works', 'word', 'worms', 'wore', 'bold', 'wooly', 'wouldn', 'wounds', 'cords', 'worid', 'hold', 'words'}}
15505
far word_len 3 pos RB search range 1
far word_len 3 pos RB search range 2
far word_len 3 pos RB search range 3
{'word_id': 15505, 'word': 'far', 'simlar_words': {'away', 'ken', 'make', 'too', 'yet', 'run', 'buy', 'save', 'more', 'rave', 'mere', 'eave', 'have', 'add', 'why', 'how', 'now', 'here', 'sly', 'see', 'say', 'take', 'ago', 'ever', 'ere', 'soar', 'not', 'back', 'find', 'get', 'very', 'late', 'let', 'pen'}}
50735
parents word_len 7 pos NNS search range 1
parents word_len 7 